## Reproducing baseline model from tensorflow tutorial
#### trained on old data: 76k of 3 channels images
#### conv1: 32 features, 5 x 5, max pool 2, 2
#### conv2: 64 features, 5 x 5, max pool 2, 2
#### fc1: 1024
#### no dropout

In [1]:
import cPickle as pickle
import gzip
import numpy as np
import os
from datetime import datetime
from time import time
import tensorflow as tf

In [3]:
with gzip.open("data/svhn_full_color_norm.pkl", "rb") as f:
  svhn_color = pickle.load(f)
  
svhn_train_data = svhn_color["train_data"]
svhn_train_labels = svhn_color["train_labels"]
svhn_test_data = svhn_color["test_data"]
svhn_test_labels = svhn_color["test_labels"]

print svhn_train_data.shape, svhn_train_labels.shape
print svhn_test_data.shape, svhn_test_labels.shape

svhn_color = None

In [74]:
print svhn_train_data.max(), svhn_test_data.max() # this data is not normalized b/w 0-1

svhn_train_data = svhn_train_data.astype(np.float32)
svhn_train_data = np.multiply(svhn_train_data, 1.0 / 255.0)
svhn_test_data = svhn_test_data.astype(np.float32)
svhn_test_data = np.multiply(svhn_test_data, 1.0 / 255.0)

print svhn_train_data.max(), svhn_train_data.mean(), svhn_test_data.max(), svhn_test_data.mean()

255 255
1.0 0.435835 1.0 0.457967


## Model Wrappers

In [69]:
def weight_variable(shape, stddev=1.0):
  initial = tf.truncated_normal(shape, stddev=stddev)
  return tf.Variable(initial)

def bias_variable(shape, default_bias=0.0):
  initial = tf.constant(default_bias, shape=shape)
  return tf.Variable(initial)

def conv_layer(layer_name, input_tensor, receptive_field, channels_in, channels_out,
              padding='SAME', stride=1, act=tf.nn.relu, decay=0.0,
              pool=True, pooler=tf.nn.max_pool, pool_size=2, pool_stride=2, pool_padding='SAME',
              batch_norm=False, training=True):
  
  with tf.name_scope(layer_name):
    with tf.name_scope('weights'):
      weights = weight_variable([receptive_field, receptive_field, channels_in, channels_out])
      
      if decay > 0:
          weight_decay = tf.multiply(tf.nn.l2_loss(weights), decay, name='weight_decay')
          tf.add_to_collection('losses', weight_decay)
  
    with tf.name_scope('biases'):
      biases = bias_variable([channels_out])
      
    with tf.name_scope('W_conv_x_plus_b'):
      preactivate = tf.nn.conv2d(input_tensor, 
                    weights, strides=[1, stride, stride, 1], padding=padding) + biases
    
    if batch_norm:
      with tf.name_scope('batchnorm'):
        normed = tf.layers.batch_normalization(preactivate, training=training)
      activations = act(normed, name='activation')
    else:
      activations = act(preactivate, name='activation')  
    
    if pool:
      max_pool = pooler(activations, ksize=[1, pool_size, pool_size, 1], 
                      strides=[1, pool_stride, pool_stride, 1],
                      padding=pool_padding)
      return max_pool
    else: 
      return activations
    
def dense_layer(layer_name, input_tensor, input_dim, output_dim, act=tf.nn.relu, decay=0.0):
  with tf.name_scope(layer_name):
    with tf.name_scope('weights'):
      weights = weight_variable([input_dim, output_dim])
      
      if decay > 0:
        weight_decay = tf.multiply(tf.nn.l2_loss(weights), decay, name='weight_decay')
        tf.add_to_collection('losses', weight_decay)

    with tf.name_scope('biases'):
      biases = bias_variable([output_dim])
    with tf.name_scope('Wx_plus_b'):
      preactivate = tf.matmul(input_tensor, weights) + biases
    activations = act(preactivate, name='activation')
    return activations
  
def flat_dimension(tensor):
  dim = 1 # Compute how many numbers we have, ignoring the batch size
  for d in tensor.get_shape()[1:].as_list():
    dim *= d
  return dim

# Normalize by subtracting per image, per channel means
def normalize_batch(batch):
  per_img_ch_means = batch.mean(axis=1)
  return batch - per_img_ch_means[:, np.newaxis, :]

# hacky next_batch
def grab_next_train_batch(batch_num, data, labels, batch_size):
  assert len(labels) == len(data)
  total_images = len(data)
  assert batch_size <= total_images
  assert batch_num <= total_batches
  
  start = (batch_num * batch_size) % (len(svhn_train_data))
  end = start + batch_size
  
  next_batch = data[start:end], labels[start:end]
  return next_batch

def grab_next_test_batch(data, labels, batch_size):
  idx = np.arange(len(data))
  np.random.shuffle(idx)
  idx = idx[:batch_size]
  
  next_batch = data[start:end], labels[start:end]
  return next_batch

In [83]:
def run():
  # RESET TF GRAPH, just in case
  tf.reset_default_graph()
  
  ### Place holders ###
  with tf.name_scope('test_train_variables'):
    batch_norm_train_mode = tf.placeholder(tf.bool) # for batch_norm mode
    tf.add_to_collection('batch_norm_train_mode', batch_norm_train_mode)
    keep_prob = tf.placeholder(tf.float32) # for drop out
    tf.add_to_collection('keep_prob', keep_prob)
    
    # Optionally track that place holders are correctly set at test and train tme
    tf.summary.scalar('batch_norm_train_mode', tf.to_int32(batch_norm_train_mode, name='ToInt32'))
    tf.summary.scalar('dropout_keep_probability', keep_prob)

  
  with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name="x-input") 
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y-input")
    tf.add_to_collection('x', x)
    tf.add_to_collection('y_', y_)
    
  ###################
  ##### Network #####
  ###################
  #   with tf.name_scope('input_reshape'):
  #     input_reshaped = tf.reshape(x, [-1, 32, 32, 3])
  #   tf.summary.image('input', input_reshaped, 5) # Optionally save 5 images to ensure reshape is working
  
  conv1 = conv_layer(layer_name='conv1', input_tensor=x, receptive_field=3, 
                      channels_in=3, channels_out=64, pool=False, pool_size=2, pool_stride=2,
                      batch_norm=False, training=batch_norm_train_mode)
  
  conv2 = conv_layer(layer_name='conv2', input_tensor=conv1, receptive_field=3, 
                    channels_in=64, channels_out=64, pool=True, pool_size=2, pool_stride=2,
                    batch_norm=False, training=batch_norm_train_mode)
  
  conv3 = conv_layer(layer_name='conv3', input_tensor=conv2, receptive_field=5, 
                    channels_in=64, channels_out=128, pool=False, pool_size=2, pool_stride=2,
                    batch_norm=False, training=batch_norm_train_mode)
  
  conv4 = conv_layer(layer_name='conv4', input_tensor=conv3, receptive_field=5, 
                    channels_in=128, channels_out=128, pool=True, pool_size=2, pool_stride=2,
                    batch_norm=False, training=batch_norm_train_mode)
  
  last_conv = conv4
  
  with tf.name_scope('last_conv_flatten'):
    conv_reshaped = tf.reshape(last_conv, [-1, flat_dimension(last_conv)])

  fc1 = dense_layer(layer_name='fc1', input_tensor=conv_reshaped, input_dim=flat_dimension(last_conv), output_dim=4096, decay=fc_decay)
  #dropped1 = tf.nn.dropout(fc1, keep_prob)
  
#   fc2 = dense_layer(layer_name='fc2', input_tensor=fc1, input_dim=2048, output_dim=2048, decay=fc_decay)
#   dropped1 = tf.nn.dropout(fc1, keep_prob)
  
  last_fc = fc1
  
  # Do not apply softmax activation yet! use the identity
  logits = dense_layer(layer_name='output', input_tensor=last_fc, input_dim=4096, output_dim=10, act=tf.identity)
  tf.add_to_collection('logits', logits)
  
  #print input_reshaped.shape
  print conv1.shape
  print conv2.shape
  print conv3.shape
  print conv4.shape
  print conv_reshaped.shape
  print fc1.shape

  ### Losses and Accuracy ###
  # Cross-Entropy Loss
  with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits)
    with tf.name_scope('total'):
      cross_entropy = tf.reduce_mean(diff)
      tf.add_to_collection('losses', cross_entropy)
  tf.summary.scalar('cross_entropy', cross_entropy)
  
  # Total loss (weight decay + cross-entropy)
  total_loss = tf.add_n(tf.get_collection('losses'), name='total_loss')

  with tf.name_scope('train'):
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(learning_rate_init, global_step, decay_steps, decay_rate)
    tf.summary.scalar('learning_rate', learning_rate)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(total_loss, global_step=global_step)

  # Other metrics
  with tf.name_scope('accuracy'):
      correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('accuracy', accuracy)

  # Might be needed for batch norm
  extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)  
  # Initializing the variables
  init = tf.global_variables_initializer()

  merged_summaries = tf.summary.merge_all()
  
  # for saving the model in the end
  saver = tf.train.Saver()

  ts = datetime.now().strftime('%Y%m%d_%H%M')
  logs_path = "logs/{}/".format(ts)
  print "-"* 70
  pwd = os.getcwd()+"/"
  print("Run the following to start tensorboard server:\n" \
        "tensorboard --logdir=/{}{}".format(pwd, logs_path))
  
  # Fill in the place holders depending on the context (training? testing?)
  def feed_dict(batch_num, mode):
    if mode == 'train':
      batch_x, batch_y = grab_next_train_batch(batch_num, svhn_train_data, svhn_train_labels, batch_size)
      keep_proba = train_keep_prob
      training_mode = True

    elif mode == 'train_no_dropout':
      batch_x, batch_y = grab_next_train_batch(batch_num, svhn_train_data, svhn_train_labels, test_batch_size)
      keep_proba = 1.0
      training_mode = False

    elif mode == 'test_no_dropout':
      batch_x = svhn_test_data[:test_batch_size]
      batch_y = svhn_test_labels[:test_batch_size]
      keep_proba = 1.0
      training_mode = False

    batch_x = normalize_batch(batch_x) # Subtract per image mean
    return {x: batch_x, y_: batch_y, keep_prob: keep_proba, batch_norm_train_mode: training_mode} # can't name values same as keys
  
  ##########################################
  ##                                      ##
  ##           Launch the graph           ##
  ##                                      ##
  ##########################################
  
  with tf.Session() as sess:
    t_start = time()
    sess.run(init)
    train_writer = tf.summary.FileWriter(logs_path + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(logs_path + '/test')
    # Training loop
    for epoch in xrange(training_epochs):
      print "Learning Rate: ", sess.run(learning_rate)
      for batch_num in xrange(total_batches):
        if batch_num % test_every == test_every - 1:

          # Record summaries and accuracy on the *test* set
          summary, acc = sess.run([merged_summaries, accuracy], feed_dict=feed_dict(batch_num, mode='test_no_dropout'))
          test_writer.add_summary(summary, epoch * total_batches + batch_num)

          # To compare against *training* set (apples to apples comparison)
          summary, _, _ = sess.run([merged_summaries, optimizer, extra_update_ops], feed_dict=feed_dict(batch_num, mode='train_no_dropout'))
          train_writer.add_summary(summary, epoch * total_batches + batch_num)

          # Print occasional progress
          print('Validation accuracy at epoch %s: batch %s: %s' % (epoch, batch_num, acc))
        else:
          sess.run([optimizer, extra_update_ops], feed_dict=feed_dict(batch_num, mode='train'))
    
    train_writer.close()
    test_writer.close()
    
    t_end = time()
    elapsed_mins = (t_end - t_start) / 60.0
    print "\nOptimization Finished! in {} minutes".format(elapsed_mins)
    
    # Save down the current model
    if not os.path.exists("models"): os.makedirs("models")
    saver.save(sess, "models/{}".format(model_name))


In [84]:
##########################################
##                                      ##
##              Parameters              ##
##                                      ##
##########################################

# Training Parameters
batch_size = 100
test_batch_size = 2000
test_every = 200

total_batches = int(len(svhn_train_data) / batch_size)

learning_rate_init = 0.001
decay_steps = total_batches / 2 #2000
decay_rate = 0.5 #0.95

training_epochs = 2



# Regularization
fc_decay = 0.0
train_keep_prob = 1.0

model_name = "baseline_old_data"

run()


(?, 32, 32, 64)
(?, 16, 16, 64)
(?, 16, 16, 128)
(?, 8, 8, 128)
(?, 8192)
(?, 4096)
----------------------------------------------------------------------
Run the following to start tensorboard server:
tensorboard --logdir=//home/ec2-user/code/cnn_svhn/logs/20170425_0419/
Learning Rate:  0.001
Validation accuracy at epoch 0: batch 199: 0.5945
Validation accuracy at epoch 0: batch 399: 0.655
Validation accuracy at epoch 0: batch 599: 0.689
Validation accuracy at epoch 0: batch 799: 0.695
Validation accuracy at epoch 0: batch 999: 0.705
Validation accuracy at epoch 0: batch 1199: 0.7085
Validation accuracy at epoch 0: batch 1399: 0.7195
Validation accuracy at epoch 0: batch 1599: 0.7145
Validation accuracy at epoch 0: batch 1799: 0.716
Validation accuracy at epoch 0: batch 1999: 0.715
Validation accuracy at epoch 0: batch 2199: 0.7175
Validation accuracy at epoch 0: batch 2399: 0.7085
Validation accuracy at epoch 0: batch 2599: 0.7235
Validation accuracy at epoch 0: batch 2799: 0.7215
Va

## Testing inference

In [ ]:
# saver = tf.train.Saver()
# x_batch, y_batch = svhn_test.next_batch(10)

# with tf.Session() as sess:
#   saver.restore(sess, "models/{}".format(model_name))
#   print "model restored"
#   #logits

#   logits = tf.get_collection("logits")[0]
#   x = tf.get_collection("x")[0]
#   y_ = tf.get_collection("y_")[0]
#   keep_prob = tf.get_collection("keep_prob")[0]
  
#   predictions = sess.run(logits, feed_dict={x: x_batch, y_: y_batch, keep_prob: 1.0})
#   y_preds = tf.argmax(predictions, 1)
#   y_actual = tf.argmax(y_batch, 1)
  
#   ## tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
#   print sess.run(y_preds)
#   print sess.run(y_actual)